In [443]:
import pandas as pd
import datetime
import numpy as np

In [650]:
# importing tweet csv files and cleaning them
tweets_df = pd.read_csv('netflix_tweets.csv')
tweets_df['time_stamp'] = pd.to_datetime(tweets_df['time_stamp'])
tweets_df['time_stamp'] = tweets_df['time_stamp'].dt.round('30min')
tweets_df_ = tweets_df.set_index(['time_stamp'])

# reading stock.csv and cleaning the df
running_stocks_thirty = pd.read_csv('netflix_stock_8_30.csv', index_col = 0)
running_stocks_thirty.index = pd.to_datetime(running_stocks_thirty.index)

# updating stock data
running_stocks_thirty = running_stocks_thirty[running_stocks_thirty.index.day >= 24].sort_index()
running_stocks_thirty['increase_decrease'] = running_stocks_thirty['4. close'] - running_stocks_thirty['1. open']
running_stocks_thirty['up_down'] = np.where(running_stocks_thirty['increase_decrease']>=0, 1, 0)
running_stocks_thirty['movement(%)'] = np.round(((running_stocks_thirty['4. close'] - running_stocks_thirty['1. open'])/running_stocks_thirty['4. close'])*100, 2)

In [655]:
running_stocks_thirty = running_stocks_thirty[running_stocks_thirty['interval']==30]
running_stocks_thirty = running_stocks_thirty[running_stocks_thirty.index.day <= 29].sort_index()

In [626]:
articles_df = pd.read_csv('netflix_article_avgs.csv', index_col = 0)
articles_df.index = pd.to_datetime(articles_df.index)
articles_df_ = articles_df[articles_df.index.day >= 24]
articles_df = articles_df_[articles_df_.index.day < 30]
articles_df = articles_df.between_time('9:30', "15:30").sort_index()

# importing tweet csv files and cleaning them
tweets_df = pd.read_csv('netflix_tweets.csv')
tweets_df['time_stamp'] = pd.to_datetime(tweets_df['time_stamp'])
tweets_df['time_stamp'] = tweets_df['time_stamp'].dt.round('30min')
tweets_df_ = tweets_df.set_index(['time_stamp'])

# reading tech indicators
tech_indic = pd.read_csv('netflix_tech_indicators.csv', index_col = 0 )
tech_indic.index = pd.to_datetime(tech_indic.index)
tech_indic_ = tech_indic[tech_indic.index.day >= 24]
tech_indic_df = tech_indic_[tech_indic_.index.day < 30]

In [550]:
# drop all the tweets before the 24th
tweets_df = tweets_df_[tweets_df_.index.day >= 24]
tweets_master_df = tweets_df.drop(['t_id', 'text', 'Unnamed: 0'], axis = 1)

In [551]:
# seperate during/after market hrs
def during_after(df):
    df_during = df.between_time('9:30', "15:30")
    df_after = df.between_time('15:30', "9:30")
    return df_during, df_after

In [552]:
open_hours_tweets_24, after_hour_tweets_24 = during_after(tweets_df_24)
open_hours_tweets_25, after_hour_tweets_25 = during_after(tweets_df_25)
open_hours_tweets_26, after_hour_tweets_26 = during_after(tweets_df_26)
open_hours_tweets_27, after_hour_tweets_27 = during_after(tweets_df_27)
open_hours_tweets_28, after_hour_tweets_28 = during_after(tweets_df_28)
open_hours_tweets_29, after_hour_tweets_29 = during_after(tweets_df_29)

In [553]:
# aggregate the scores 
def aggregate(df):
    if df.empty != True:
        sent_avg = df.groupby(df.index).agg({'vader_sentiment': 'mean'}).rename(columns = {'vader_sentiment': 'sentiment_avg_30'}).shift()
        sent_count = df.groupby(df.index).agg({'sentiment': 'value_counts'}).unstack().rename(columns = {'sentiment': 'sentiment_30'}).shift()
        df_ = pd.concat([sent_avg, sent_count], axis = 1)
        df_ini = df_.fillna(df_.mean())
        df_hour_i = df_ini.shift().rolling(window=2, center = True, min_periods=2).mean().rename(columns = {'sentiment_avg_30': 'sentiment_avg_60'})
        df_hour = df_hour_i.fillna(df_hour_i.mean())
        df_two_hours_i = df_ini.shift(2).rolling(window=4, center = True, min_periods=4).mean().rename(columns = {'sentiment_avg_30': 'sentiment_avg_120'})
        df_two_hours = df_two_hours_i.fillna(df_two_hours_i.mean())
        df_all = pd.concat([df_ini, df_hour, df_two_hours], axis = 1)
        return df_all

In [554]:
open_hours_tweets_24 = aggregate(open_hours_tweets_24)
open_hours_tweets_25 = aggregate(open_hours_tweets_25)
open_hours_tweets_26 = aggregate(open_hours_tweets_26)
open_hours_tweets_27 = aggregate(open_hours_tweets_27)
open_hours_tweets_28 = aggregate(open_hours_tweets_28)
open_hours_tweets_29 = aggregate(open_hours_tweets_29)

In [556]:
df_morning = tweets_23_night.between_time('0:00', "9:30")

In [557]:
# seperate night-before and morning-day-of market hrs
def night_morning(df):
    df_night = df.between_time('15:30', "0:00")
    df_morning = df.between_time('0:00', "9:30")
    df_night_avg = df_night.groupby(df_night.index.day).mean()
    df_night_ct = df_night.groupby(df_night.index.day).agg({'sentiment': 'value_counts'}).unstack()
    night_df = pd.concat([df_night_avg, df_night_ct], axis = 1)
    if df_morning.empty != True:
        df_morning_avg = df_morning.groupby(df_morning.index.day).mean()
        df_morning_ct = df_morning.groupby(df_morning.index.day).agg({'sentiment': 'value_counts'}).unstack()
        morning_df = pd.concat([df_morning_avg, df_morning_ct], axis = 1)
    else:
        morning_df = df_morning
    if len(night_df.columns) == 4:
        night_df.columns = ['vader_sentiment_night', 'sent_neg, night_before', 'sent_neu, night_before', 'sent_pos, night_before']
    if len(morning_df.columns) == 4:
        morning_df.columns = ['vader_sentiment_morning', 'sent_neg, morning', 'sent_neu, morning', 'sent_pos, morning']
    pd.to_datetime(morning_df.index)
    pd.to_datetime(night_df.index)
    return night_df, morning_df

In [558]:
tweets_df_23 = tweets_df_[tweets_df_.index.day == 23]
tweets_23_df = tweets_df_23.drop(['t_id', 'text', 'Unnamed: 0'], axis = 1)
tweets_23_night = tweets_23_df.between_time('15:30', "9:30")
after_hour_tweets_24_night, after_hour_tweets_23_morning = night_morning(tweets_23_night)

In [559]:
after_hour_tweets_25_night, after_hour_tweets_24_morning = night_morning(after_hour_tweets_24)
after_hour_tweets_26_night, after_hour_tweets_25_morning = night_morning(after_hour_tweets_25)
after_hour_tweets_27_night, after_hour_tweets_26_morning = night_morning(after_hour_tweets_26)
after_hour_tweets_28_night, after_hour_tweets_27_morning = night_morning(after_hour_tweets_27)
after_hour_tweets_29_night, after_hour_tweets_28_morning = night_morning(after_hour_tweets_28)
after_hour_tweets_30_night, after_hour_tweets_29_morning = night_morning(after_hour_tweets_29)

In [560]:
# update the index of the night df to the next day
after_hour_tweets_24_night.index = [24]
after_hour_tweets_25_night.index = [25]
after_hour_tweets_26_night.index = [26]
after_hour_tweets_27_night.index = [27]
after_hour_tweets_28_night.index = [28]
after_hour_tweets_29_night.index = [29]

In [561]:
night_morning_24 = pd.concat([after_hour_tweets_24_night, after_hour_tweets_24_morning], axis = 1)
night_morning_25 = pd.concat([after_hour_tweets_25_night, after_hour_tweets_25_morning], axis = 1)
night_morning_26 = pd.concat([after_hour_tweets_26_night, after_hour_tweets_26_morning], axis = 1)
night_morning_27 = pd.concat([after_hour_tweets_27_night, after_hour_tweets_27_morning], axis = 1)
night_morning_28 = pd.concat([after_hour_tweets_28_night, after_hour_tweets_28_morning], axis = 1)
night_morning_29 = pd.concat([after_hour_tweets_29_night, after_hour_tweets_29_morning], axis = 1)

In [562]:
list_of_columns = list(night_morning_24.columns)
def concat_main_night_morning(open_hour_df, night_morning_df):
    for i in range(0,len(list_of_columns)):
        open_hour_df[list_of_columns[i]] = np.full((len(open_hour_df),1), night_morning_df[list_of_columns[i]].values[0])

In [563]:
# open_hours_tweets_24 = open_hours_tweets_25.copy()
# open_hours_tweets_27 = open_hours_tweets_25.copy()
# open_hours_tweets_28 = open_hours_tweets_25.copy()
# open_hours_tweets_29 = open_hours_tweets_25.copy()

In [565]:
concat_main_night_morning(open_hours_tweets_24, night_morning_24) # replace later
concat_main_night_morning(open_hours_tweets_25, night_morning_25)
concat_main_night_morning(open_hours_tweets_26, night_morning_26)
concat_main_night_morning(open_hours_tweets_27, night_morning_27)
concat_main_night_morning(open_hours_tweets_28, night_morning_28)
concat_main_night_morning(open_hours_tweets_29, night_morning_29)

In [593]:
# open_hours_tweets_24.columns = ['sentiment_avg_30','coun_neg, 30', 'coun_neu, 30','coun_pos, 30', 'sentiment_avg_60', 'coun_neg, 60', 'coun_neu, 60','coun_pos, 60','sentiment_avg_120', 'coun_neg, 120', 'coun_neu, 120','coun_pos, 120', 'sentiment_night_before_avg', 'count_neg, night_before', 'count_neu, night_before', 'count_pos, night_before','sentiment_morning_avg', 'count_neg, morning', 'count_neu, morning', 'count_pos, morning']
# open_hours_tweets_27.columns = ['sentiment_avg_30','coun_neg, 30', 'coun_neu, 30','coun_pos, 30', 'sentiment_avg_60', 'coun_neg, 60', 'coun_neu, 60','coun_pos, 60','sentiment_avg_120', 'coun_neg, 120', 'coun_neu, 120','coun_pos, 120', 'sentiment_night_before_avg', 'count_neg, night_before', 'count_neu, night_before', 'count_pos, night_before','sentiment_morning_avg', 'count_neg, morning', 'count_neu, morning', 'count_pos, morning']
# open_hours_tweets_28.columns = ['sentiment_avg_30','coun_neg, 30', 'coun_neu, 30','coun_pos, 30', 'sentiment_avg_60', 'coun_neg, 60', 'coun_neu, 60','coun_pos, 60','sentiment_avg_120', 'coun_neg, 120', 'coun_neu, 120','coun_pos, 120', 'sentiment_night_before_avg', 'count_neg, night_before', 'count_neu, night_before', 'count_pos, night_before','sentiment_morning_avg', 'count_neg, morning', 'count_neu, morning', 'count_pos, morning']
# open_hours_tweets_29.columns = ['sentiment_avg_30','coun_neg, 30', 'coun_neu, 30','coun_pos, 30', 'sentiment_avg_60', 'coun_neg, 60', 'coun_neu, 60','coun_pos, 60','sentiment_avg_120', 'coun_neg, 120', 'coun_neu, 120','coun_pos, 120', 'sentiment_night_before_avg', 'count_neg, night_before', 'count_neu, night_before', 'count_pos, night_before','sentiment_morning_avg', 'count_neg, morning', 'count_neu, morning', 'count_pos, morning']
# open_hours_tweets_25.columns = ['sentiment_avg_30','coun_neg, 30', 'coun_neu, 30','coun_pos, 30', 'sentiment_avg_60', 'coun_neg, 60', 'coun_neu, 60','coun_pos, 60','sentiment_avg_120', 'coun_neg, 120', 'coun_neu, 120','coun_pos, 120', 'sentiment_night_before_avg', 'count_neg, night_before', 'count_neu, night_before', 'count_pos, night_before','sentiment_morning_avg', 'count_neg, morning', 'count_neu, morning', 'count_pos, morning']
# open_hours_tweets_26.columns = ['sentiment_avg_30','coun_neg, 30', 'coun_neu, 30','coun_pos, 30', 'sentiment_avg_60', 'coun_neg, 60', 'coun_neu, 60','coun_pos, 60','sentiment_avg_120', 'coun_neg, 120', 'coun_neu, 120','coun_pos, 120', 'sentiment_night_before_avg', 'count_neg, night_before', 'count_neu, night_before', 'count_pos, night_before','sentiment_morning_avg', 'count_neg, morning', 'count_neu, morning', 'count_pos, morning']

In [594]:
# open_hours_tweets_24['sentiment_avg_30'] = 0
# open_hours_tweets_24['coun_neg, 30'] = 0
# open_hours_tweets_24['coun_neu, 30'] = 0
# open_hours_tweets_24['coun_pos, 30'] = 0
# open_hours_tweets_24['sentiment_avg_60'] = 0
# open_hours_tweets_24['coun_neg, 60'] = 0
# open_hours_tweets_24['coun_neu, 60'] = 0
# open_hours_tweets_24['coun_pos, 60'] = 0
# open_hours_tweets_24['sentiment_avg_120'] = 0
# open_hours_tweets_24['coun_neg, 120'] = 0
# open_hours_tweets_24['coun_neu, 120'] = 0
# open_hours_tweets_24['coun_pos, 120'] = 0

In [595]:
# open_hours_tweets_27['sentiment_avg_30'] = 0
# open_hours_tweets_27['coun_neg, 30'] = 0
# open_hours_tweets_27['coun_neu, 30'] = 0
# open_hours_tweets_27['coun_pos, 30'] = 0
# open_hours_tweets_27['sentiment_avg_60'] = 0
# open_hours_tweets_27['coun_neg, 60'] = 0
# open_hours_tweets_27['coun_neu, 60'] = 0
# open_hours_tweets_27['coun_pos, 60'] = 0
# open_hours_tweets_27['sentiment_avg_120'] = 0
# open_hours_tweets_27['coun_neg, 120'] = 0
# open_hours_tweets_27['coun_neu, 120'] = 0
# open_hours_tweets_27['coun_pos, 120'] = 0

In [596]:
# open_hours_tweets_28['sentiment_avg_30'] = 0
# open_hours_tweets_28['coun_neg, 30'] = 0
# open_hours_tweets_28['coun_neu, 30'] = 0
# open_hours_tweets_28['coun_pos, 30'] = 0
# open_hours_tweets_28['sentiment_avg_60'] = 0
# open_hours_tweets_28['coun_neg, 60'] = 0
# open_hours_tweets_28['coun_neu, 60'] = 0
# open_hours_tweets_28['coun_pos, 60'] = 0
# open_hours_tweets_28['sentiment_avg_120'] = 0
# open_hours_tweets_28['coun_neg, 120'] = 0
# open_hours_tweets_28['coun_neu, 120'] = 0
# open_hours_tweets_28['coun_pos, 120'] = 0

In [598]:
# open_hours_tweets_24_col = pd.DataFrame(pd.DatetimeIndex(['2018-08-24 14:30:00', '2018-08-24 15:00:00',
#                '2018-08-24 15:30:00']))
# open_hours_tweets_24_col.columns = ['time_stamp']
# open_hours_tweets_24_col = open_hours_tweets_24_col.set_index(['time_stamp'])
# open_hours_tweets_24.index = open_hours_tweets_24_col.index
# open_hours_tweets_27_col = pd.DataFrame(pd.DatetimeIndex(['2018-08-27 14:30:00', '2018-08-27 15:00:00',
#                '2018-08-27 15:30:00']))
# open_hours_tweets_27_col.columns = ['time_stamp']
# open_hours_tweets_27_col = open_hours_tweets_27_col.set_index(['time_stamp'])
# open_hours_tweets_27.index = open_hours_tweets_27_col.index
# open_hours_tweets_28_col = pd.DataFrame(pd.DatetimeIndex(['2018-08-28 14:30:00', '2018-08-28 15:00:00',
#                '2018-08-28 15:30:00']))
# open_hours_tweets_28_col.columns = ['time_stamp']
# open_hours_tweets_28_col = open_hours_tweets_28_col.set_index(['time_stamp'])
# open_hours_tweets_28.index = open_hours_tweets_28_col.index
# open_hours_tweets_29_col = pd.DataFrame(pd.DatetimeIndex(['2018-08-29 14:30:00', '2018-08-29 15:00:00',
#                '2018-08-29 15:30:00']))
# open_hours_tweets_29_col.columns = ['time_stamp']
# open_hours_tweets_29_col = open_hours_tweets_29_col.set_index(['time_stamp'])
# open_hours_tweets_29.index = open_hours_tweets_29_col.index

In [601]:
tweets_master = pd.concat([open_hours_tweets_24, open_hours_tweets_25, open_hours_tweets_26, open_hours_tweets_27, open_hours_tweets_28, open_hours_tweets_29])
# tweets_master.columns = ['sentiment_avg_30','coun_neg, 30', 'coun_neu, 30','coun_pos, 30', 'sentiment_avg_60', 'coun_neg, 60', 'coun_neu, 60','coun_pos, 60','sentiment_avg_120', 'coun_neg, 120', 'coun_neu, 120','coun_pos, 120', 'sentiment_night_before_avg', 'count_neg, night_before', 'count_neu, night_before', 'count_pos, night_before','sentiment_morning_avg', 'count_neg, morning', 'count_neu, morning', 'count_pos, morning']
# take out weekend
tweets_master = tweets_master[(tweets_master.index.day != 25) & (tweets_master.index.day != 26)]

In [618]:
# new_index = articles_df.index
# tweets_master = tweets_master.reindex(new_index)

In [658]:
# put stock and tweet data together
tweets_stocks = pd.concat([running_stocks_thirty, tech_indic_df, tweets_master, articles_df], axis = 1)

In [660]:
tweets_stocks = tweets_stocks.drop(['interval'], axis = 1)

In [665]:
tweets_stocks

,1. open,2. high,3. low,4. close,5. volume,increase_decrease,up_down,movement(%),MACD_Signal,MACD,...,one day the-new-york-times pos avg,one day business-insider pos avg,one day techcrunch pos avg,one day wired pos avg,one day abc-news neu avg,one day reuters neu avg,one day the-new-york-times neu avg,one day business-insider neu avg,one day techcrunch neu avg,one day wired neu avg
date,,,,,,,,,,,,,,,,,,,,,
2018-08-24 09:30:00,345.9900,349.6500,344.5500,347.9498,2394400,1.9598,1,0.56,0.8865,0.6506,...,0.0910,0.101250,NaN,NaN,NaN,NaN,0.865000,0.843750,NaN,NaN
2018-08-24 10:00:00,348.2000,350.3411,348.2000,349.3570,1353317,1.1570,1,0.33,0.9487,1.1973,...,0.0910,0.101250,NaN,NaN,NaN,NaN,0.865000,0.843750,NaN,NaN
2018-08-24 10:30:00,349.5000,352.0800,349.4000,352.0283,1267592,2.5283,1,0.72,1.1239,1.8251,...,0.0910,0.101250,NaN,NaN,NaN,NaN,0.865000,0.843750,NaN,NaN
2018-08-24 11:00:00,352.0930,352.7500,350.3000,351.1900,815234,-0.9030,0,-0.26,1.3450,2.2293,...,0.1010,0.101250,NaN,0.1100,NaN,NaN,0.852000,0.843750,NaN,0.882
2018-08-24 11:30:00,351.3450,353.2800,350.9000,353.0600,699483,1.7150,1,0.49,1.6100,2.6697,...,0.1010,0.101250,NaN,0.1100,NaN,NaN,0.852000,0.843750,NaN,0.882
2018-08-24 12:00:00,353.0826,354.2529,353.0777,354.2250,741462,1.1424,1,0.32,1.9034,3.0773,...,0.1010,0.101250,NaN,0.1100,NaN,NaN,0.852000,0.843750,NaN,0.882
2018-08-24 12:30:00,354.1900,356.4111,353.7000,356.2861,1003701,2.0961,1,0.59,2.2279,3.5260,...,0.1010,0.101250,NaN,0.1100,NaN,NaN,0.852000,0.843750,NaN,0.882
2018-08-24 13:00:00,356.2092,356.8100,354.7100,355.4778,907084,-0.7314,0,-0.21,2.5369,3.7729,...,0.1010,0.101250,NaN,0.1100,NaN,NaN,0.852000,0.843750,NaN,0.882
2018-08-24 13:30:00,355.3251,357.2700,355.2000,356.8935,718453,1.5684,1,0.44,2.8368,4.0362,...,0.1100,0.203000,NaN,0.1100,NaN,NaN,0.854500,0.764000,NaN,0.882


In [666]:
tweets_stocks.to_csv('netflix_tweets_stock.csv')